# Ticket/Request Triage with Structured Outputs

**Goal:** Turn messy emails / Jira / ServiceNow tickets into clean, reliable JSON your apps can route automatically.

**No GPU required.**

What you’ll build:
- A **Pydantic schema** for triage outputs
- A single-call **triage function** using the OpenAI **Responses API**
- A small **batch triage** pipeline + basic QA checks



## 1. Setup and Installation

If you’re in Colab, run the install cell below. If you’re local, install once in your virtual env.

You’ll also need an environment variable named `OPENAI_API_KEY`.


In [ ]:
!pip -q install --upgrade openai pydantic pandas

## 2. Imports + API client

We’ll use the OpenAI Python SDK and Pydantic to guarantee a schema-shaped response.


In [ ]:
import os
from enum import Enum
from typing import List, Optional

import pandas as pd
from pydantic import BaseModel, Field, conint, confloat

from openai import OpenAI

# --- API key check ---
if not os.getenv("OPENAI_API_KEY"):
    raise EnvironmentError(
        "Missing OPENAI_API_KEY. Set it in your environment (or Colab Secrets) and re-run."
    )

client = OpenAI()


## 3. Define a triage schema (Pydantic)

This schema is what your downstream system expects.
Use enums to prevent invalid categories/priority values.


In [ ]:
class Priority(str, Enum):
    P0 = "P0"   # outage / critical
    P1 = "P1"   # high urgency
    P2 = "P2"   # normal
    P3 = "P3"   # low

class Category(str, Enum):
    ACCESS = "access"
    HARDWARE = "hardware"
    SOFTWARE = "software"
    NETWORK = "network"
    WEBSITE = "website"
    DATA = "data"
    FACILITIES = "facilities"
    PATRON_SERVICES = "patron_services"
    OTHER = "other"

class TicketTriage(BaseModel):
    # A short one-liner you could show in a queue
    short_summary: str = Field(..., description="One sentence summary of the request.")

    priority: Priority = Field(..., description="Urgency/impact priority P0–P3.")
    category: Category = Field(..., description="Routing category.")

    # Routing and next steps
    routing_team: str = Field(..., description="Which team should own this (e.g., 'IT Helpdesk', 'Web', 'Network').")
    recommended_next_step: str = Field(..., description="Concrete next action for the owning team.")

    # Optional details that help triage quickly
    affected_systems: List[str] = Field(default_factory=list, description="List of systems/services impacted.")
    requires_human_followup: bool = Field(..., description="True if a human must follow up before action.")
    missing_info: List[str] = Field(default_factory=list, description="What info is missing that blocks action.")

    # A lightweight self-reported confidence score for human review routing
    confidence: confloat(ge=0, le=1) = Field(..., description="0–1 confidence in the triage classification.")


## 4. Sample tickets (synthetic NYPL-style)

These are realistic but **fake** examples (no personal data).


In [ ]:
tickets = [
    {
        "id": "INC-10421",
        "source": "ServiceNow",
        "text": "Public Wi-Fi is down on the 3rd floor at Stavros Niarchos Foundation Library. Staff can't connect and patrons are complaining. Started ~30 minutes ago."
    },
    {
        "id": "JIRA-882",
        "source": "Jira",
        "text": "Website search results intermittently return 500 errors. Looks like spikes around 2pm. Affects /search endpoint. Please investigate and add monitoring."
    },
    {
        "id": "EMAIL-55",
        "source": "Email",
        "text": "Hi IT, I can't log into the staff portal (SSO). It says 'account locked'. I need access for a program registration today."
    },
    {
        "id": "INC-10488",
        "source": "ServiceNow",
        "text": "A self-checkout kiosk is stuck on the boot screen after a power flicker. Branch: Harlem. Please advise if we should reboot or dispatch support."
    },
    {
        "id": "REQ-2201",
        "source": "ServiceNow",
        "text": "Request: weekly CSV export of event registrations by borough for program ops. Needed by Monday morning."
    },
    {
        "id": "EMAIL-90",
        "source": "Email",
        "text": "Patron reports they received the wrong due-date notice in email. Can we verify messaging templates and recent deploy?"
    },
]

df = pd.DataFrame(tickets)
df


## 5. Single-call triage with Structured Outputs

We’ll:
- Provide the ticket text
- Ask for a `TicketTriage` object
- Let the SDK parse + validate the output for us

This uses `client.responses.parse(...)` with a Pydantic model.

In [ ]:
TRIAGE_SYSTEM = """You are an IT + library operations triage assistant.
Return a JSON object that matches the provided schema exactly.

Rules:
- If the issue is an outage impacting many users or core service: choose P0.
- If urgent for a time-sensitive program or many users: choose P1.
- If normal: P2. If low impact or informational: P3.
- If key info is missing, list it in missing_info and set requires_human_followup=true.
- Use concise, actionable language.
"""


def triage_ticket(ticket_text: str, model: str = "gpt-4o-mini") -> TicketTriage:
    response = client.responses.parse(
        model=model,
        input=[
            {"role": "system", "content": TRIAGE_SYSTEM},
            {"role": "user", "content": ticket_text},
        ],
        text_format=TicketTriage,
    )
    return response.output_parsed


# Try one:
triage_ticket(df.loc[0, "text"])


## 6. Batch triage + basic QA checks

In production you’ll want:
- A simple batch loop
- Basic validation (already handled by schema)
- Guardrails: flag low-confidence items for humans


In [ ]:
def batch_triage(df: pd.DataFrame, text_col: str = "text", model: str = "gpt-4o-mini") -> pd.DataFrame:
    rows = []
    for _, r in df.iterrows():
        triage = triage_ticket(r[text_col], model=model)
        rows.append({**r.to_dict(), **triage.model_dump()})
    return pd.DataFrame(rows)

triaged = batch_triage(df)
triaged[["id","priority","category","routing_team","confidence","short_summary"]]


### 6.1 Human review routing

A common workflow: auto-route only if confidence is high and missing_info is empty.


In [ ]:
AUTO_THRESHOLD = 0.75

triaged["auto_route_ok"] = (triaged["confidence"] >= AUTO_THRESHOLD) & (triaged["missing_info"].apply(len) == 0)

triaged[["id","priority","category","confidence","auto_route_ok","missing_info"]]


## 7. Exercises

### EXERCISE 1: Add SLA + escalation fields
1. Extend the schema with:
- `sla_hours: int` (0–168)
- `escalate: bool`
2. Update the prompt rules so P0/P1 map to shorter SLAs.

### EXERCISE 2: Add a deterministic override layer
Before calling the model, implement a rule:
- If text contains "account locked" or "SSO" → category = access

### EXERCISE 3: CSV in / CSV out
Load a CSV of tickets (id,text) and write a triaged CSV.


In [ ]:
# EXERCISE STARTER CELL (write your solution here)

# 1) Extend schema
# 2) Update triage prompt
# 3) Add override rules
# 4) Batch CSV

pass
